In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
envs = load_dotenv('.env', override=True)

import os
from rich import print
from pathlib import Path
from langchain.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, multi_context_question_prompt
from ragas.metrics import (
answer_correctness,
answer_relevancy,
faithfulness,
context_recall,
context_precision)
from ragas import evaluate

In [2]:
api_key = os.environ['OPENAI_API_KEY']

In [3]:
data_dir = '/Users/americanthinker/Downloads/Data/acled_reports/'
data_dir

'/Users/americanthinker/Downloads/Data/acled_reports/'

In [4]:
bad_files = [path for path in Path(data_dir).iterdir() if not path.name.endswith('txt')]

In [6]:
loader = DirectoryLoader(data_dir)

In [7]:
documents = loader.load()

In [9]:
len(documents)
for doc in documents:
    doc.metadata['filename'] = doc.metadata['source']

In [11]:
gen_llm = ChatOpenAI(api_key=api_key, model='gpt-3.5-turbo')

In [12]:
critic_llm = ChatOpenAI(api_key=api_key, model='gpt-4')

In [13]:
embeddings =OpenAIEmbeddings(api_key=api_key)

In [14]:
generator = TestsetGenerator.from_langchain(gen_llm, critic_llm, embeddings, chunk_size=350)

In [5]:
import json
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
with open('/Users/americanthinker/Downloads/english.json') as f:
    data = json.loads(f.read())

/opt/anaconda3/envs/vsa/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for explodinggradients/amnesty_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/explodinggradients/amnesty_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [11]:
amnesty_qa

DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [31]:
result = evaluate(amnesty_qa['eval'], metrics=[context_precision, 
                                 faithfulness,
                                 answer_relevancy,
                                 context_recall,
                                 answer_correctness
                                ],
                 )

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
type(result)

ragas.evaluation.Result

In [32]:
df = result.to_pandas()

In [33]:
df

,question,ground_truth,answer,contexts,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ...",1.0,NaN,0.987989,1.0,0.333333
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[- Fossil fuel companies, whether state or pri...",1.0,0.416667,0.959061,1.0,0.450000
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The private companies responsible for the mos...,1.0,1.000000,0.987002,1.0,0.461538
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,[Amnesty International called on its vast netw...,1.0,0.200000,0.932881,1.0,0.800000
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,[Amnesty International recommends that the Spe...,1.0,0.100000,0.993736,1.0,0.333333
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,[Amnesty International has therefore created t...,1.0,1.000000,0.999208,1.0,1.000000
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[26. The Act raises serious questions about it...,1.0,1.000000,0.972157,1.0,1.000000
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,[- The victims of gross human rights violation...,1.0,1.000000,0.972391,1.0,0.666667
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,1.0,0.000000,0.907087,1.0,0.000000
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...,1.0,1.000000,0.929678,1.0,0.500000


In [34]:
df.loc[19].values

array(['When did the government of Qatar start repealing restrictions on migrant workers?',
       'The government of Qatar started repealing restrictions on migrant workers between 2018 and 2020.',
       'The government of Qatar started repealing restrictions on migrant workers in 2017.',
       array(["Between 2018 and 2020, the government took important steps towards tackling two central pillars of this system by repealing for most migrant workers the requirement to obtain from their employer an 'exit permit’ to leave the country, and a ‘no -objection certificate’ (NOC) to change jobs.\nOther barriers to workers changing jobs freely have emerged over the last few years, limiting the impact of the reforms and maintaining the status quo.\nThese include the unwritten – but often spoken – need for workers to secure “release papers”, a resignation letter, or any equivalent of an NOC, from their current employers in order to successfully move jobs."],
             dtype=object)          

In [30]:
answer_correctness.weights

[1.0, 0]